In [18]:
import pandas as pd
import numpy as np

# FTHG = Full Time Home Team Goals
# FTAG = Full Time Away Team Goals
# FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)

fields = ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"]

df = pd.read_csv("data/Ligue1_game_stats.csv", usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])

# Rename the columns for clarity: 
df = df.rename(columns={'FTHG': 'Home_goals', 'FTAG': 'Away_goals', 'FTR': 'Result'})
# Home_goals = Full Time Home Team Goals
# Away_goals = Full Time Away Team Goals
# Result = Full Time Result (H=Home Win, D=Draw, A=Away Win)


df.head()


/var/folders/42/4qk9d5xn1zb22gk9hbqpxdwh0000gn/T/ipykernel_15907/3495449552.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


,Date,HomeTeam,AwayTeam,Home_goals,Away_goals,Result
0,2025-08-15,Rennes,Marseille,1,0,H
1,2025-08-16,Lens,Lyon,0,1,A
2,2025-08-16,Monaco,Le Havre,3,1,H
3,2025-08-16,Nice,Toulouse,0,1,A
4,2025-08-17,Brest,Lille,3,3,D


In [19]:
# Dataset with only Marseille games

df_om_games = df[(df["HomeTeam"] == "Marseille") | (df["AwayTeam"] == "Marseille")].copy()


df_om_games.head()

df_om_games["OM_goals_scored"] = np.where(df_om_games['HomeTeam'] == 'Marseille', df_om_games["Home_goals"], df_om_games["Away_goals"])
df_om_games["OM_goals_conceded"] = np.where(df_om_games['HomeTeam'] == 'Marseille', df_om_games["Away_goals"], df_om_games["Home_goals"])
df_om_games["is_OM_win"] = np.where(df_om_games['HomeTeam'] == 'Marseille', df_om_games["Result"] == "H", df_om_games["Result"] == "A")
df_om_games["is_Draw"] = df_om_games["Result"] == "D"


df_om_games.head(10)

df_om_games = df_om_games.assign(goals_scored_last_3_games=lambda d: d["OM_goals_scored"].rolling(3).sum().shift())
df_om_games = df_om_games.assign(goals_conceded_last_3_games=lambda d: d["OM_goals_conceded"].rolling(3).sum().shift())

df_om_games

,Date,HomeTeam,AwayTeam,Home_goals,Away_goals,Result,OM_goals_scored,OM_goals_conceded,is_OM_win,is_Draw,goals_scored_last_3_games,goals_conceded_last_3_games
0,2025-08-15,Rennes,Marseille,1,0,H,0,1,False,False,NaN,NaN
10,2025-08-23,Marseille,Paris FC,5,2,H,5,2,True,False,NaN,NaN
26,2025-08-31,Lyon,Marseille,1,0,H,0,1,False,False,NaN,NaN
27,2025-09-12,Marseille,Lorient,4,0,H,4,0,True,False,5.0,4.0
44,2025-09-22,Marseille,Paris SG,1,0,H,1,0,True,False,9.0,3.0
45,2025-09-26,Strasbourg,Marseille,1,2,A,2,1,True,False,5.0,1.0
55,2025-10-04,Metz,Marseille,0,3,A,3,0,True,False,7.0,1.0
66,2025-10-18,Marseille,Le Havre,6,2,H,6,2,True,False,6.0,1.0
75,2025-10-25,Lens,Marseille,2,1,H,1,2,False,False,11.0,3.0
85,2025-10-29,Marseille,Angers,2,2,D,2,2,False,True,10.0,4.0


In [ ]:
# TODO, add number of rest days before game - pour ça il va falloir aussi rajouter les matchs joués en champions league.
# add Forme récente : points pris sur les 5 derniers matchs, différence de buts, série de victoires/défaites
# A voir si on peut ajouter: les joueurs blessés. 
# Add current rank
# Add Elo score

## Construction of a model for the 2024-2025 season


In [20]:
# Loading the data of the previous year for training the model. 
df_2024 = pd.read_csv("data/Ligue1_2024-2025.csv", usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])

# Rename the columns for clarity: 
df_2024 = df_2024.rename(columns={'FTHG': 'Home_goals', 'FTAG': 'Away_goals', 'FTR': 'Result'})

df_2024.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'Home_goals', 'Away_goals', 'Result'], dtype='object')

## Construction of a model for the 2024-2025 season


In [211]:
# Loading the data of the previous year for training the model. 
df_2024 = pd.read_csv("data/Ligue1_2024-2025.csv", usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])

# Rename the columns for clarity: 
df_2024 = df_2024.rename(columns={'FTHG': 'Home_goals', 'FTAG': 'Away_goals', 'FTR': 'Result'})

df_2024.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'Home_goals', 'Away_goals', 'Result'], dtype='object')

### Creating an ELO score, which compute a rank of the teams using a numerical score

In [212]:
# Defining the ELO function: 
import pandas as pd
import numpy as np

def expected_score(rating_A, rating_B):
    """Odds that A wins against B"""
    return 1 / (1 + 10 ** ((rating_B - rating_A) / 400))

def update_elo(rating_A, rating_B, score_A, K=30):
    """
    Update Elo ratings after a match
    score_A : 1 = A wins, 0.5 = draw, 0 = A loses
    We chose K=30 as of now based on common practice for this hyper_parameter in football 
    """
    
    expected_A = expected_score(rating_A, rating_B)
    expected_B = 1 - expected_A
    
    new_rating_A = round(rating_A + K * (score_A - expected_A),0)
    new_rating_B = round(rating_B + K * ((1 - score_A) - expected_B),0)
    
    return new_rating_A, new_rating_B


In [235]:

# Implementation to the 2024 dataset

# Initialisation
teams = pd.concat([df_2024['HomeTeam']]).unique()
elo = {team: 1500 for team in teams}  # Initial score
elo_A, elo_B = [], []

# Iteration
for i, row in df_2024.iterrows():
    team_A, team_B = row['HomeTeam'], row['AwayTeam']
    res = row['Result']

    # Results
    if res == 'H': score_A, score_B = 1, 0
    elif res == 'A': score_A, score_B = 0, 1
    else: score_A, score_B = 0.5, 0.5
    
    # Save before update
    elo_A.append(elo[team_A])
    elo_B.append(elo[team_B])
    
    # Update
    new_A, new_B = update_elo(elo[team_A], elo[team_B], score_A)
    elo[team_A], elo[team_B] = new_A, new_B

print(elo_A)
df_2024['Elo_Home'] = elo_A
df_2024['Elo_Away'] = elo_B
df_2024['Diff_Elo'] = df_2024['Elo_Home'] - df_2024['Elo_Away']




[1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1515.0, 1485.0, 1515.0, 1485.0, 1515.0, 1500.0, 1485.0, 1500.0, 1515.0, 1471.0, 1471.0, 1486.0, 1499.0, 1529.0, 1471.0, 1500.0, 1486.0, 1529.0, 1455.0, 1528.0, 1484.0, 1544.0, 1483.0, 1530.0, 1499.0, 1485.0, 1529.0, 1486.0, 1496.0, 1498.0, 1518.0, 1542.0, 1459.0, 1473.0, 1457.0, 1490.0, 1455.0, 1555.0, 1526.0, 1486.0, 1555.0, 1485.0, 1461.0, 1512.0, 1497.0, 1538.0, 1461.0, 1511.0, 1486.0, 1492.0, 1473.0, 1532.0, 1514.0, 1501.0, 1579.0, 1488.0, 1476.0, 1565.0, 1456.0, 1456.0, 1494.0, 1457.0, 1449.0, 1475.0, 1453.0, 1525.0, 1538.0, 1521.0, 1438.0, 1504.0, 1501.0, 1545.0, 1559.0, 1542.0, 1592.0, 1504.0, 1448.0, 1471.0, 1477.0, 1429.0, 1479.0, 1544.0, 1498.0, 1510.0, 1487.0, 1536.0, 1444.0, 1409.0, 1473.0, 1520.0, 1553.0, 1614.0, 1523.0, 1453.0, 1504.0, 1541.0, 1510.0, 1453.0, 1536.0, 1505.0, 1447.0, 1460.0, 1624.0, 1413.0, 1447.0, 1532.0, 1492.0, 1542.0, 1547.0, 1508.0, 1549.0, 1533.0, 1477.0, 1523.0, 1444.0, 1456.0, 1451.0, 1495.0, 1

In [236]:
# Verification
elo_rank = {k: v for k, v in sorted(elo.items(), key=lambda item: item[1])}
print(elo_rank)
print(df_2024)

{'Montpellier': 1302.0, 'Reims': 1405.0, 'St Etienne': 1413.0, 'Angers': 1438.0, 'Le Havre': 1448.0, 'Nantes': 1454.0, 'Rennes': 1461.0, 'Toulouse': 1474.0, 'Auxerre': 1478.0, 'Brest': 1512.0, 'Lens': 1523.0, 'Lyon': 1551.0, 'Lille': 1564.0, 'Monaco': 1566.0, 'Strasbourg': 1572.0, 'Nice': 1577.0, 'Marseille': 1579.0, 'Paris SG': 1683.0}
           Date    HomeTeam     AwayTeam  Home_goals  Away_goals Result  \
0    16/08/2024    Le Havre     Paris SG           1           4      A   
1    17/08/2024       Brest    Marseille           1           5      A   
2    17/08/2024       Reims        Lille           0           2      A   
3    17/08/2024      Monaco   St Etienne           1           0      H   
4    18/08/2024     Auxerre         Nice           2           1      H   
..          ...         ...          ...         ...         ...    ...   
301  17/05/2025      Nantes  Montpellier           3           0      H   
302  17/05/2025        Nice        Brest           6         

### Features engeneering

In [238]:
# Compute current form of the teams

# Initialisation
teams = pd.concat([df_2024['HomeTeam']]).unique()
perf_last_3_games = {team: 0 for team in teams} # Number of points won in the last 3 games
perf_A, perf_B = [], []
memory = {team: [0,0,0] for team in teams}

# Iteration
for i, row in df_2024.iterrows():
    team_A, team_B = row['HomeTeam'], row['AwayTeam']
    res = row['Result']

    # Results
    if res == 'H': score_A, score_B = 3, 0
    elif res == 'A': score_A, score_B = 0, 3
    else: score_A, score_B = 1, 1

    # Save before update
    perf_A.append(perf_last_3_games[team_A])
    perf_B.append(perf_last_3_games[team_B])
    
    # Update memory
    memory[team_A]=(memory[team_A][1:])
    memory[team_A].append(score_A)
    memory[team_B]=(memory[team_B][1:])
    memory[team_B].append(score_B)

    # Update perf
    perf_last_3_games[team_A] = sum(memory[team_A])
    perf_last_3_games[team_B] = sum(memory[team_B])

df_2024['current_form_Home'] = perf_A
df_2024['current_form_Away'] = perf_B

In [239]:
# Compute recent goals scored and conceded for each team

#**This code does not work** , I'll fix it later

matches_2024 = df_2024.copy()  # rename to avoid conflict

# Home perspective
home_df = matches_2024[["Date", "HomeTeam", "Home_goals", "Away_goals"]].copy()
home_df.rename(columns={
    "HomeTeam": "Team",
    "Home_goals": "goals_scored",
    "Away_goals": "goals_conceded"
}, inplace=True)

# Away perspective
away_df = matches_2024[["Date", "AwayTeam", "Away_goals", "Home_goals"]].copy()
away_df.rename(columns={
    "AwayTeam": "Team",
    "Away_goals": "goals_scored",
    "Home_goals": "goals_conceded"
}, inplace=True)

# Combine both perspectives
team_df = pd.concat([home_df, away_df], ignore_index=True)
team_df.sort_values(["Team", "Date"], inplace=True)

# Rolling stats per team
team_df["goals_scored_last_3"] = team_df.groupby("Team")["goals_scored"].rolling(3).sum().shift().reset_index(level=0, drop=True)
team_df["goals_conceded_last_3"] = team_df.groupby("Team")["goals_conceded"].rolling(3).sum().shift().reset_index(level=0, drop=True)

# Merge back to original matches_2024
matches_2024 = matches_2024.merge(
    team_df[["Date", "Team", "goals_scored_last_3", "goals_conceded_last_3"]].rename(columns={
        "Team": "HomeTeam",
        "goals_scored_last_3": "Home_goals_scored_last_3",
        "goals_conceded_last_3": "Home_goals_conceded_last_3"
    }),
    on=["Date", "HomeTeam"],
    how="left"
)

matches_2024 = matches_2024.merge(
    team_df[["Date", "Team", "goals_scored_last_3", "goals_conceded_last_3"]].rename(columns={
        "Team": "AwayTeam",
        "goals_scored_last_3": "Away_goals_scored_last_3",
        "goals_conceded_last_3": "Away_goals_conceded_last_3"
    }),
    on=["Date", "AwayTeam"],
    how="left"
)


### First training not especially on OM, but for all teams

**ATTENTION**: We have Nans in the goals: I drop them for now, we'll se how we can handle them

Update: We don't use goals as of now, so no need

In [240]:
# Drop the Nan
#df_2024.isna().sum()
#df_2024_clean = df_2024.dropna()
#df_2024_clean.isna().sum()
#df_2024_clean.columns


In [241]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier

features = ['Elo_Home', 'Elo_Away', 'current_form_Home','current_form_Away']

X = df_2024[features]
y = df_2024['Result']

tscv = TimeSeriesSplit(n_splits=5) #We use TimeSeriesSplit for defining the different sample in order to avoid training the model with data posterior to the test dataset.
model = RandomForestClassifier()

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.score(X_test, y_test))

importances = model.feature_importances_
feature_importance_dict = dict(zip(features, importances))
print(feature_importance_dict)

0.49019607843137253
0.5098039215686274
0.49019607843137253
0.5490196078431373
0.5098039215686274
{'Elo_Home': np.float64(0.35447786066667514), 'Elo_Away': np.float64(0.3383937128285204), 'current_form_Home': np.float64(0.14301069002237213), 'current_form_Away': np.float64(0.16411773648243233)}


0.49019607843137253
0.37254901960784315
0.47058823529411764
0.5882352941176471
0.49019607843137253

The results are insanely bad. Cannot get worse at least

In [198]:
# TODO, add number of rest days before game - pour ça il va falloir aussi rajouter les matchs joués en champions league.
# A voir si on peut ajouter: les joueurs blessés. 
# Réussir à intégrer les buts marqués dans les derniers matchs
# Pour l'instant, le modèle est très mauvais. A voir si des modèles performent mieux que des arbres de décisions (sûrement)

### Dataset with only Marseille's game : **USELESS FOR NOW, AS WE TRAINED THE MODEL FOR ALL TEAMS**

In [243]:
df_om_games_2024 = df_2024[(df_2024["HomeTeam"] == "Marseille") | (df_2024["AwayTeam"] == "Marseille")].copy()


df_om_games_2024["OM_goals_scored"] = np.where(df_om_games_2024['HomeTeam'] == 'Marseille', df_om_games_2024["Home_goals"], df_om_games_2024["Away_goals"])
df_om_games_2024["OM_goals_conceded"] = np.where(df_om_games_2024['HomeTeam'] == 'Marseille', df_om_games_2024["Away_goals"], df_om_games_2024["Home_goals"])
df_om_games_2024["is_OM_win"] = np.where(df_om_games_2024['HomeTeam'] == 'Marseille', df_om_games_2024["Result"] == "H", df_om_games_2024["Result"] == "A")
df_om_games_2024["ELO OM"] = np.where(df_om_games_2024['HomeTeam'] == 'Marseille', df_om_games_2024["Elo_Home"], df_om_games_2024["Elo_Away"])
df_om_games_2024["ELO Opponent"] = np.where(df_om_games_2024['HomeTeam'] == 'Marseille', df_om_games_2024["Elo_Away"], df_om_games_2024["Elo_Home"])
df_om_games_2024["OM_at_home"] = 1 * df_2024["HomeTeam"] == "Marseille"


df_om_games_2024.head(10)

df_om_games_2024 = df_om_games_2024.assign(OM_goals_scored_last_3_games=lambda d: d["OM_goals_scored"].rolling(3).sum().shift())
df_om_games_2024 = df_om_games_2024.assign(OM_goals_conceded_last_3_games=lambda d: d["OM_goals_conceded"].rolling(3).sum().shift())

df_om_games_2024

,Date,HomeTeam,AwayTeam,Home_goals,Away_goals,Result,Elo_Home,Elo_Away,Diff_Elo,current_form_Home,current_form_Away,OM_goals_scored,OM_goals_conceded,is_OM_win,ELO OM,ELO Opponent,OM_at_home,OM_goals_scored_last_3_games,OM_goals_conceded_last_3_games
1,17/08/2024,Brest,Marseille,1,5,A,1500.0,1500.0,0.0,0,0,5,1,True,1500.0,1500.0,False,NaN,NaN
17,25/08/2024,Marseille,Reims,2,2,D,1515.0,1485.0,30.0,3,0,2,2,False,1515.0,1485.0,True,NaN,NaN
21,31/08/2024,Toulouse,Marseille,1,3,A,1499.0,1514.0,-15.0,2,4,3,1,True,1514.0,1499.0,False,NaN,NaN
28,14/09/2024,Marseille,Nice,2,0,H,1528.0,1500.0,28.0,7,4,2,0,True,1528.0,1500.0,True,10.0,4.0
44,22/09/2024,Lyon,Marseille,2,3,A,1490.0,1542.0,-52.0,4,7,3,2,True,1542.0,1490.0,False,7.0,3.0
53,29/09/2024,Strasbourg,Marseille,1,0,H,1497.0,1555.0,-58.0,2,9,0,1,False,1555.0,1497.0,False,8.0,3.0
54,04/10/2024,Marseille,Angers,1,1,D,1538.0,1449.0,89.0,6,2,1,1,False,1538.0,1449.0,True,5.0,3.0
71,20/10/2024,Montpellier,Marseille,0,5,A,1449.0,1534.0,-85.0,3,4,5,0,True,1534.0,1449.0,False,4.0,4.0
80,27/10/2024,Marseille,Paris SG,0,3,A,1545.0,1578.0,-33.0,4,7,0,3,False,1545.0,1578.0,True,6.0,2.0
89,03/11/2024,Nantes,Marseille,1,2,A,1479.0,1531.0,-52.0,1,4,2,1,True,1531.0,1479.0,False,6.0,4.0
